In [1]:
# Common library imports
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image
import random
import openslide as op
import openslide.deepzoom as deepzoom
import cv2

# Imports from TensorFlow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D,\
                                    MaxPooling2D,GlobalAveragePooling2D, BatchNormalization                                    
from tensorflow.keras import utils, regularizers
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import utils

# Setting a random seed for reproducible results
np.random.seed(42)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3

In [2]:
# Setting targets
TARGETS = ['normal', 'squamous', 'adenocarcinoma']

# Creating a path to data directory
train_dir = '/Train/'
test_dir = '/Test/'

In [3]:
training_data = []
def create_training_data():
    for target in TARGETS:
        path = os.path.join(train_dir, target)
        class_num = TARGETS.index(target)
        for img in os.listdir(path):
            try:
                # Open the file and deep zoom generator.
                slide = op.OpenSlide(os.path.join(path, img))
                tile = op.deepzoom.DeepZoomGenerator(osr = slide, tile_size = 4320, overlap = 0)
                
                # one off of maximal zoom
                level = tile.level_count - 2 
                x, y = tile.level_tiles[level]

                # Creating an array
                img_array = np.array(tile.get_tile(level, ((x//2)-1, (y//2)-1))).astype('float32')
                training_data.append([img_array, class_num])
            
            except:
                continue
                
create_training_data()

In [4]:
testing_data = []
def create_testing_data():
    for target in TARGETS:
        path = os.path.join(test_dir, target)
        class_num = TARGETS.index(target)
        for img in os.listdir(path):
            try:
                # Open the file and deep zoom generator.
                slide = op.OpenSlide(os.path.join(path, img))
                tile = op.deepzoom.DeepZoomGenerator(osr = slide, tile_size = 4320, overlap = 0)
                
                # 1 off of maximal zoom
                level = tile.level_count - 2 
                x, y = tile.level_tiles[level]

                # Creating an array
                img_array = np.array(tile.get_tile(level, ((x//2)-1, (y//2)-1))).astype('float32')
                testing_data.append([img_array, class_num])
            
            except:
                continue
            
create_testing_data()

In [5]:
X_train = []
y_train = []

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)
    
X_train = np.array(X_train).reshape(-1, 4320, 4320, 3)

In [6]:
X_test = []
y_test = []

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)
    
X_test = np.array(X_test).reshape(-1, 4320, 4320, 3)

In [ ]:
model = Sequential()

model.add(Conv2D(
    filters = 16,
    kernel_size = (3,3),
    activation = "relu",
    input_shape = (4320, 4320, 3)
))

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(
    filters = 64, 
    kernel_size = (2,2),
    activation = "relu"))

model.add(MaxPooling2D(
    pool_size = (2,2)
))

model.add(Flatten())

model.add(Dense(
    128, 
    activation = "relu",
    kernel_regularizer = regularizers.l2(0.5)
))

model.add(Dropout(0.5))

model.add(Dense(
    3, 
    activation = "softmax",
    kernel_regularizer = regularizers.l2(0.5)
))

es = EarlyStopping(
    monitor = "val_loss",
    min_delta = 0,
    patience = 5
)

model.compile(
    # Sparse_categorical_crossentropy will convert indexed targets to categorical targets
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy']
)

history = model.fit(
        X_train,
        y_train,
        batch_size = 10,
        validation_data = (X_test, y_test),
        epochs = 10,
        callbacks = [es],
        verbose = 1,
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 59 samples, validate on 11 samples
